In [1]:
import pandas as pd
import seaborn as sns

# K-Means (k-nearest neightbor algorithm, kNN)

- train.csv は sotck_id, time_id と予測すべき target (volatility) が載っているデータ

In [2]:
from sklearn.cluster import KMeans
# making agg features

train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


- ピボット処理（データ成形）を行う。インデックスに `time_id`、カラム名に `stock_id`、表のデータとして `target` を書き込む。
- stock_id （株の銘柄）同士の相関を取る。pandas では変数は列単位に並んでいるので、相関を取る変数は列同士であることに留意。

In [3]:
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')
train_p.head()

stock_id,0,1,2,3,4,5,6,7,8,9,...,115,116,118,119,120,122,123,124,125,126
time_id,,,,,,,,,,,,,,,,,,,,,
5,0.004136,0.006340,0.001848,0.005300,0.004468,0.006234,0.007651,0.003624,0.010036,0.007291,...,0.004267,0.007944,0.003336,0.002571,0.003035,0.004862,0.002942,0.004112,0.001919,0.008067
11,0.001445,0.002099,0.000806,0.002774,0.001852,0.002562,0.004670,0.002458,0.002291,0.002529,...,0.001382,0.002469,0.002030,0.000839,0.001271,0.002095,0.001518,0.001891,0.001123,0.003965
16,0.002168,0.002456,0.001581,0.002986,0.002213,0.003253,0.004303,0.002178,0.001841,0.003299,...,0.001949,0.002195,0.003410,0.002569,0.002137,0.001893,0.002131,0.002428,0.001548,0.003161
31,0.002195,0.002807,0.001599,0.004437,0.002256,0.003072,0.005401,0.002149,0.003997,0.003696,...,0.002035,0.002298,0.005674,0.002115,0.001734,0.003509,0.001078,0.002182,0.001251,0.003593
62,0.001747,0.004312,0.001503,0.003408,0.002102,0.002824,0.004562,0.002203,0.003923,0.003689,...,0.002459,0.003704,0.003914,0.001549,0.001470,0.002151,0.001253,0.002382,0.001324,0.003496


In [4]:
corr = train_p.corr()
corr

stock_id,0,1,2,3,4,5,6,7,8,9,...,115,116,118,119,120,122,123,124,125,126
stock_id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.798070,0.841460,0.647471,0.852778,0.853056,0.665597,0.835231,0.521143,0.703246,...,0.853943,0.822769,0.849980,0.857607,0.858696,0.814953,0.888719,0.759692,0.860547,0.807063
1,0.798070,1.000000,0.694983,0.707773,0.791688,0.801733,0.708319,0.750894,0.723175,0.778223,...,0.813933,0.839468,0.760083,0.741288,0.799673,0.845167,0.819863,0.753699,0.730829,0.880584
2,0.841460,0.694983,1.000000,0.534588,0.835739,0.813608,0.517432,0.885969,0.415829,0.601032,...,0.841273,0.752961,0.851451,0.890355,0.826682,0.766011,0.890501,0.704816,0.943743,0.722100
3,0.647471,0.707773,0.534588,1.000000,0.650228,0.663158,0.769600,0.627592,0.593529,0.747987,...,0.688678,0.766134,0.670604,0.614615,0.729989,0.757407,0.645228,0.764252,0.575922,0.715880
4,0.852778,0.791688,0.835739,0.650228,1.000000,0.847807,0.650958,0.833176,0.551661,0.723698,...,0.848445,0.825794,0.845132,0.835639,0.841994,0.821867,0.868835,0.746809,0.846285,0.811599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0.814953,0.845167,0.766011,0.757407,0.821867,0.823992,0.723928,0.805659,0.692158,0.790123,...,0.851971,0.894657,0.804392,0.770963,0.839737,1.000000,0.816422,0.817551,0.786192,0.855556
123,0.888719,0.819863,0.890501,0.645228,0.868835,0.868485,0.642725,0.878215,0.531187,0.696700,...,0.869953,0.821938,0.853334,0.908795,0.877944,0.816422,1.000000,0.776829,0.905294,0.823923
124,0.759692,0.753699,0.704816,0.764252,0.746809,0.761487,0.732119,0.757622,0.579541,0.757330,...,0.770445,0.814858,0.782857,0.747317,0.838834,0.817551,0.776829,1.000000,0.745062,0.767322


In [5]:
corr.values.shape

(112, 112)

## kNN

- 112次元のベクトルをクラスタリングしている（7クラスに分類しているが、これは恣意的か？）
- `labels_` で、入力の112個の変数がどのクラスタに分類されているかを返してくれる

In [18]:
# stock_id 
ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)
print(ids)
print(kmeans.labels_)
len(kmeans.labels_)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            115, 116, 118, 119, 120, 122, 123, 124, 125, 126],
           dtype='int64', name='stock_id', length=112)
[1 0 4 2 1 1 2 4 6 2 1 0 4 4 1 1 1 2 4 4 4 0 1 1 3 1 1 4 3 4 3 4 4 1 3 3 4
 3 4 1 4 1 4 4 1 0 4 4 1 0 0 3 3 3 2 0 2 4 1 4 4 1 4 1 0 3 3 0 3 0 6 5 3 3
 0 1 2 0 3 3 3 4 1 1 0 2 3 3 1 0 1 4 4 4 4 4 1 3 1 0 1 4 1 0 1 4 1 0 4 0 4
 0]


112

## クラス分類の結果を加工

- -1/+1 は、stock_id が0から始まるので：
    - `ids * kmeans_labels_ == n_class` で抽出するときには、+1 して掛け算がゼロにならないようにする
    - `(x-1)` で実際のstock_id になおして

In [7]:
l = []
for n_class in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n_class)) if x > 0] )

In [19]:
# ex. クラス番号「1」のものを True にする条件
kmeans.labels_==1

array([ True, False, False, False,  True,  True, False, False, False,
       False,  True, False, False, False,  True,  True,  True, False,
       False, False, False, False,  True,  True, False,  True,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False,  True, False, False,  True,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False,  True, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False,  True,  True, False, False, False, False,  True, False,
        True, False, False, False, False, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
       False, False, False, False])

In [13]:
(ids+1) * (kmeans.labels_ == 1)

Int64Index([  1,   0,   0,   0,   5,   6,   0,   0,   0,   0,
            ...
            116,   0, 119,   0, 121,   0,   0,   0,   0,   0],
           dtype='int64', name='stock_id', length=112)

In [ ]:
mat = []
matTest = []

for n, ind in enumerate(l):
    print(ind)
    newDf = train.loc[train['stock_id'].isin(ind)]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = f'{n}c1'
    mat.append(newDf)
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = f'{n}c1'
    matTest.append(newDf)

In [ ]:
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()